In [1]:
import tkinter as tk
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
import utils as ut
from importlib import reload
ut = reload(ut)
UtilsRender = ut.UtilsRender

In [3]:
# Define the payload
payload = {
    "capRadius": 20,
    "configs": [
        {"capIndex": 0, "capCenter": {"x": 250, "y": 250}},
        {"capIndex": 1, "capCenter": {"x": 200, "y": 250}},
        {"capIndex": 2, "capCenter": {"x": 550, "y": 550}},
    ],
    "arrow": {"capIndex": 0, "arrow_length": 100, "angle": 20},
}

In [4]:
response = {
  "motion": [
    {
      "t": 0,
      "configs": [
        { "capIndex": 0, "capCenter": { "x": 250, "y": 250 }},
        { "capIndex": 1, "capCenter": { "x": 200, "y": 250 }},
        { "capIndex": 2, "capCenter": { "x": 550, "y": 555 }}
      ]
    },
    {
      "t": 1,
      "configs": [
        { "capIndex": 0, "capCenter": { "x": 255, "y": 250 }},
        { "capIndex": 1, "capCenter": { "x": 205, "y": 250 }},
        { "capIndex": 2, "capCenter": { "x": 550, "y": 555 }}
      ]
    },
    {
      "t": 2,
      "configs": [
        { "capIndex": 0, "capCenter": { "x": 260, "y": 250 }},
        { "capIndex": 1, "capCenter": { "x": 210, "y": 250 }},
        { "capIndex": 2, "capCenter": { "x": 550, "y": 559 }}
      ]
    },
    {
      "t": 3,
      "configs": [
        { "capIndex": 0, "capCenter": { "x": 370, "y": 250 }},
        { "capIndex": 1, "capCenter": { "x": 220, "y": 250 }},
        { "capIndex": 2, "capCenter": { "x": 550, "y": 570 }}
      ]
    }
  ]
}

In [5]:
UtilsRender.render_motion_json(payload, response)

# Simplify problem

In [11]:
w = 1920
h = 1080
delay = 100

# Define the payload
payload = {
    "capRadius": 20,
    "configs": [
        # {"capIndex": 0, "capCenter": {"x": w//5, "y": h//2}},
        {"capIndex": 0, "capCenter": {"x": 30, "y": h//2}},
        {"capIndex": 1, "capCenter": {"x": int(w*2//3), "y": h//2}},
    ],
    "arrow": {"capIndex": 0, "arrow_length": 100, "angle": 0},
}

# Get the coordinates from payload to create an array 2D of positions
pos = np.array([[config["capCenter"]["x"], config["capCenter"]["y"]] for config in payload["configs"]])
idx = np.array([config["capIndex"] for config in payload["configs"]])

# Radius
cap_radius = payload["capRadius"]

# Arrow properties detection
arrow = payload["arrow"]
arrow_idx = arrow["capIndex"]
arrow_velocity = arrow["arrow_length"]
arrow_angle = arrow["angle"]
arrow_angle_rad = np.radians(arrow_angle)



In [12]:
pos

array([[  30,  540],
       [1280,  540]])

In [9]:

ur = UtilsRender(cap_radius=cap_radius, window_size=(w, h), delay=delay)

# Get the coordinates from payload to create an array 2D of positions
pos = np.array([[config["capCenter"]["x"], config["capCenter"]["y"]] for config in payload["configs"]])
idx = np.array([config["capIndex"] for config in payload["configs"]])

# Initialize parameters
initial_velocity = 2
min_velocity = 0.1
move_idx = 0

# Next Velocity
def next_velocity(velocity):
    arr_vel_threshold = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 6, 7, 8, 9, 10])
    scales_threshold = np.array([100, 80, 35, 30, 25, 30, 40, 45, 50, 50, 50, 50, 50, 50])

    for i, vel in enumerate(arr_vel_threshold):
        if velocity < vel:
            scale = scales_threshold[i]
            break
        else:
            scale = scales_threshold[-1]
    delta = -1./scale
    velocity += delta
    velocity = np.round(velocity, 4)
    return velocity

# Initialize velocity
velocity = initial_velocity

# Initialize list to store positions
positions = [pos.copy()]
velocities = [velocity]

# Calculate direction vector from angle
direction = np.array([math.cos(arrow_angle_rad), -math.sin(arrow_angle_rad)])

# Start movement loop
t = -1
while velocity >= min_velocity:
    t += 1
    # Calculate new position
    move = direction * velocity
    new_pos = pos[move_idx] + move

    # Check for collision with field edges
    if new_pos[0] - cap_radius < 0 or new_pos[0] + cap_radius > w or new_pos[1] - cap_radius < 0 or new_pos[1] + cap_radius > h:
        print("Collision with field edges")
        break

    # Check for collision with other caps
    for i, cap_pos in enumerate(pos):
        if i != move_idx:
            arrow_length = np.linalg.norm(new_pos - cap_pos)
            if arrow_length < 2 * cap_radius:
                if arrow_length < velocity:
                    velocity = arrow_length
                    move = direction * velocity
                    new_pos = pos[move_idx] + move
                print(f"Collision t={t} with other cap")
                break
    else:
        # Update position and velocity
        pos[move_idx] = new_pos
        print("--------------------")
        print("Timestamp", t)
        print("Velocity", velocity)
        new_velocity = next_velocity(velocity=velocity)
        
        print("New velocity", new_velocity)
        print("--------------------")

        # Store new position
        positions.append(pos.copy())

        # Store velocity to velocities
        velocities.append(new_velocity)
        velocity = new_velocity

        continue

    break

# Convert positions to numpy array
positions = np.array(positions)


--------------------
Timestamp 0
Velocity 2
New velocity 1.96
--------------------
--------------------
Timestamp 1
Velocity 1.96
New velocity 1.9267
--------------------
--------------------
Timestamp 2
Velocity 1.9267
New velocity 1.8934
--------------------
--------------------
Timestamp 3
Velocity 1.8934
New velocity 1.8601
--------------------
--------------------
Timestamp 4
Velocity 1.8601
New velocity 1.8268
--------------------
--------------------
Timestamp 5
Velocity 1.8268
New velocity 1.7935
--------------------
--------------------
Timestamp 6
Velocity 1.7935
New velocity 1.7602
--------------------
--------------------
Timestamp 7
Velocity 1.7602
New velocity 1.7269
--------------------
--------------------
Timestamp 8
Velocity 1.7269
New velocity 1.6936
--------------------
--------------------
Timestamp 9
Velocity 1.6936
New velocity 1.6603
--------------------
--------------------
Timestamp 10
Velocity 1.6603
New velocity 1.627
--------------------
-------------------

In [10]:
ur.render_motion(positions, add_delay=5)

In [307]:
ur.debug_motion(positions, print_velocities=velocities)

In [120]:
ur.render_snapshot(positions[-1])

In [125]:
n_caps = pos.shape[0]

In [126]:
pos

array([[1239,  540],
       [1280,  540]])

In [127]:
x = np.array(
    [[1239,  540],
    [1280,  540],
    [700,  210],
    [700,  250]]
)